# Counting Rubik's Snake Loops

Loop is a shape where tail and head are adjacent.

Let's define four numbers for $n$-wedge Rubik's Snake:

* L1(n) - number of formulas (i.e. strings of characters 0,1,2,3 of length $n-1$) that describe shape that are loops.
* L2(n) - number of formulas, deduplicated "up to reversal" that describe loop.
* L3(n) - number of loops (described by n-character formulas) up to cyclic shifts.
* L4(n) - number of loops up to cyclic shifts and reversals.

L4 is the most meaningful number. It is the number of different loops if we look at loops as sets of wedges (ignoring where the tail-head connection is), and consider two loops the same if they can be matched by translation and rotation.

All these values are zeros for odd $n$, and for n=2. So we will be computing values for n=4,6,8,10...

Below are values for n from 4 to 22 that I got.

In [1]:
L1_GOLDEN = [1,8,16,280,2229,20720,226000,2293422,24965960,275633094]
L2_GOLDEN = [1,5,10,145,1129,10405,113113,1147142,12484285,137821030]
L3_GOLDEN = [1,3,3,31,196,1509,14191,127681,1248963,12531157]
L4_GOLDEN = [1,2,3,18,112,777,7198,64056,625584,6267820]

These are all normalized loops (in sense of definition L4) for n=4,6,8,10.

In [2]:
GOLDEN_UNIQUE_LOOPS = {
  4: {'2222'},
  6: {'131313', '123123'},
  8: {'01230321', '00220022', '12321232'},
  10: {'0131201312', '0021331132', '0021013032', '0013211022', '0231303132', 
       '0121323022', '0230320323', '0113133022', '0012300123', '0113203312', 
       '0123023212', '0131202131', '0013100313', '0021330313', '0120121021', 
       '0013101132', '0022033213', '0231302313'}
}

Code computing the loops.

We enumerate all loops using meet-in the middle, then do deduplication.

In [3]:
import numba
import numpy as np
import time

# Efficient formula encoding.
def encode_formula_as_int(s):
  assert all(48<=ord(c)<=51 for c in s)
  n = len(s)
  return sum((ord(s[i])-48)<<(2*(n-1-i)) for i in range(len(s)))

def decode_formula(code, length):
  return ''.join(str((code>>(2*i))%4) for i in range(length))[::-1]
 
@numba.jit("i8(i8,i8)", inline="always")
def reverse_encoded_formula(code, length):
  ans = 0
  for i in range(length):
    ans = (ans<<2) + (code>>(2*i))%4
  return ans
  
@numba.jit("i8(i8,i8)", inline="always")  
def min_cyclic_shift(code, length):
  ans = code
  l = 2*(length-1)
  for i in range(length - 1):
    code = (code>>2) + ((code&3)<<l)
    if code<ans: ans=code
  return ans 

# Test.
for n in range(1,7):
  for code in range(4**n):
    formula = decode_formula(code, n)
    assert len(formula)==n
    assert encode_formula_as_int(formula) == code
    assert decode_formula(reverse_encoded_formula(code,n),n) == formula[::-1]
    min_shift = decode_formula(min_cyclic_shift(code,n), n)
    expected_min_shift = min(formula[i:] + formula[:i] for i in range(n))
    assert min_shift == expected_min_shift

In [4]:
from rubiks_snake import RubiksSnakeCounter, _push_wedge,_pop_wedge,_push_next_wedge_if_can, \
    BOX_SIZE,CENTER_COORD,FACE_IDS_TO_WEDGE_ID,DY


# Enumerates all shapes.
@numba.jit("(i8[:],i8[:],i8,i8[:],i8[:])")
def _enumerate_shapes_rec(wedges, cubes, cur_formula, formulas, last_wedges):
    last_wedge_index = wedges[0]
    last_wedge = wedges[wedges[0]]
    if wedges[0] == 1:
      next_pos = formulas[0]
      formulas[formulas[0]] = cur_formula
      last_wedges[formulas[0]] = last_wedge
      formulas[0]+=1
      return

    for rot in range(4):
      if _push_next_wedge_if_can(rot, wedges, cubes):
        _enumerate_shapes_rec(wedges, cubes, (cur_formula<<2)+rot, formulas, last_wedges)
        _pop_wedge(wedges, cubes)
        
        
# Enumerates shapes of length n, their formulas have length n-1.
def enumerate_shapes(n, first_wedge_faces=(0,3)):  
  assert n<=20
  # This should be class members.
  wedges = np.zeros(n + 1, dtype=np.int64)
  wedges[0] = n + 1
  cubes = np.zeros(BOX_SIZE ** 3, dtype=np.int64)
  _push_wedge(CENTER_COORD, FACE_IDS_TO_WEDGE_ID[first_wedge_faces], wedges, cubes)  # Initial wedge.

  num_shapes = RubiksSnakeCounter.S[n]
  formulas = np.zeros(num_shapes+1, dtype=np.int64) 
  last_wedges = np.zeros_like(formulas)
  formulas[0] = 1
  
  _enumerate_shapes_rec(wedges, cubes, 0, formulas, last_wedges)
  
  assert formulas[0] == 1 + num_shapes
  return formulas[1:], last_wedges[1:]



@numba.jit("i8(i8,i8,i8)", inline="always")
def concat_encoded_formulas(code1, code2, length2):
  return (code1<<(2*length2)) + code2


# Tries to add wedges to tail, instructed by rotations in formula.
# Formula has given length(>0) and encoded by formula encoding convention.
# Returns number of added wedges.
# If result == n, means all added successfully.
# If result <n, only this much were added and then got spacial conflict.
@numba.jit("i8(i8,i8,i8[:],i8[:])", inline="always")
def _add_wedges_from_formula_while_can(formula_code, formula_length, wedges, cubes) -> int:
  k = 2*(formula_length-1)
  for i in range(formula_length):
    rot = (formula_code>>k)&3
    k -= 2
    if not _push_next_wedge_if_can(rot, wedges, cubes):
      return i
  return formula_length


@numba.jit("(i8,i8[:],i8[:])", inline="always")
def _pop_n_wedges(n, wedges, cubes):
    for _ in range(n):
        _pop_wedge(wedges, cubes)

@numba.jit("i8(i8,i8)", inline="always")
def normalize_encoded_loop_L2(f, n):
  f_without_last = f//4
  return min(f_without_last, reverse_encoded_formula(f_without_last, n-1))

@numba.jit("i8(i8,i8)", inline="always")
def normalize_encoded_loop_L3(f, n):
  return min_cyclic_shift(f, n)
        
@numba.jit("i8(i8,i8)", inline="always")
def normalize_encoded_loop_L4(f, n):
  rev = reverse_encoded_formula(f, n)
  return min(min_cyclic_shift(f, n), min_cyclic_shift(rev, n))

INIT_WEDGE = FACE_IDS_TO_WEDGE_ID[(0,3)]
@numba.jit()#"Tuple(int64, set(int64))(i8,i8[:],i8[:],i8[:],i8[:])")
def _enumerate_loops_helper(n, formulas_1, last_wedges_1, formulas_2, last_wedges_2):
  half_shapes_count = len(formulas_1)
  assert len(formulas_2) == half_shapes_count
  m = n // 2
    
  # Manual map to store mapping wedge_id -> List[formula]. 
  linked_list_2 = np.full((half_shapes_count,), -1)
  wedge_id_to_formula_pos_2 = numba.typed.Dict.empty(
    key_type=numba.types.int64,
    value_type=numba.types.int64)
  
  for i in range(half_shapes_count):
    last_wedge = last_wedges_2[i]
    if last_wedge in wedge_id_to_formula_pos_2:
      linked_list_2[i] = wedge_id_to_formula_pos_2[last_wedge] 
    wedge_id_to_formula_pos_2[last_wedge]=i  
  
  wedges = np.zeros(n + 1, dtype=np.int64)
  wedges[0] = n + 1
  cubes = np.zeros(BOX_SIZE ** 3, dtype=np.int64)
  _push_wedge(CENTER_COORD, INIT_WEDGE, wedges, cubes)  # Initial wedge.

  loop_formulas_count = 0 
  #normalized_loops = set()
  
  for i in range(half_shapes_count):
    lookup_key = last_wedges_1[i] ^ 16
    if lookup_key not in wedge_id_to_formula_pos_2:
      continue
    formula1 = formulas_1[i]
    assert _add_wedges_from_formula_while_can(formula1, m, wedges, cubes) == m
    ll_pos = wedge_id_to_formula_pos_2[lookup_key]
    while ll_pos != -1:
      formula2 = formulas_2[ll_pos]
      ll_pos = linked_list_2[ll_pos]
      formula2_rev = reverse_encoded_formula(formula2, m)      
      added_from_f2 = _add_wedges_from_formula_while_can(formula2_rev, m, wedges, cubes) 
      #assert added_from_f2 <= m-1
      if added_from_f2 == m-1:
        #assert wedges[wedges[0]]>>6 == CENTER_COORD - DY
        loop_formula = concat_encoded_formulas(formula1, formula2_rev, m)
        # At this point all loop_formulas are enumerated.
        loop_formulas_count += 1
        #normalized_loops.add(normalize_encoded_loop_L4(loop_formula, n))
      _pop_n_wedges(added_from_f2, wedges, cubes)    
    _pop_n_wedges(m, wedges, cubes)
    assert wedges[0] == n
    
  return loop_formulas_count #, normalized_loops
  
# Enumerates all formulas correspdning to loops, no dedup whatsoever.
def enumerate_all_loop_formulas(n):
  t0 = time.time()
  if n<4 or n%2==1:
    return np.zeros(0, dtype=np.int64)
  formulas_1, last_wedges_1 = enumerate_shapes(n//2+1, first_wedge_faces=(0,3))
  formulas_2, last_wedges_2 = enumerate_shapes(n//2+1, first_wedge_faces=(3,0))
  print("Enumerated half-shapes in %f" % (time.time() - t0))
  return _enumerate_loops_helper(n, formulas_1, last_wedges_1, formulas_2, last_wedges_2)
    
@numba.jit("(i8,i8[:],i8[:],i8[:],i8[:])")
def _coalesce_loops(n, l1, l2, l3, l4):
  for i in range(len(l1)):
    f = l1[i]
    f_without_last = f//4
    l2[i] = min(f_without_last, reverse_encoded_formula(f_without_last, n-1))
    f_rev = reverse_encoded_formula(f, n)
    f1 = min_cyclic_shift(f, n)
    l3[i] = f1
    l4[i] = min(f1, min_cyclic_shift(f_rev, n))
  
@numba.jit("i8(i8[:])")
def _count_unique(a):
  a.sort()
  ans = 1
  for i in range(1, len(a)):
    if a[i]!=a[i-1]:
      ans +=1
  return ans
  
def analyze_loops(n):
  t0=time.time()
  print("n=",n)  
  L1 = enumerate_all_loop_formulas(n)
  print("Enumerated all loop formulas", time.time()-t0)
  print("L1=", L1)
  #L4 = len(l4)
  #print("L4=", L4)
  
 # if n<=10:
    #unique_loops_str = set(decode_formula(f, n) for f in l4)
    #assert GOLDEN_UNIQUE_LOOPS[n] == unique_loops_str
  if n<=22:
    assert L1 == L1_GOLDEN[n//2-2]
    #assert L4 == L4_GOLDEN[n//2-2]
    
  print("Time %fs" % (time.time()-t0))
  print("", flush=True)

#for n in range(4,17,2):
#  analyze_loops(n)
analyze_loops(18)

n= 18
Enumerated half-shapes in 0.010787
Enumerated all loop formulas 4.371681451797485
L1= 2293422
Time 4.371751s

